In [1]:
from qiskit import *
from qiskit.opflow import X, Y, Z, I, PauliTrotterEvolution
from qiskit.opflow import (StateFn, Zero, One, Plus, Minus, H,
                           DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)
from qiskit.circuit import Parameter, Gate
from qiskit.quantum_info.operators import Operator, Pauli
import numpy as np
import math

In [2]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ionq import IonQProvider

provider = IonQProvider()

In [3]:
provider.backends()

[<IonQSimulatorBackend('ionq_simulator')>, <IonQQPUBackend('ionq_qpu')>]

In [4]:
backend = provider.get_backend("ionq_simulator")

In [5]:
f = 0.5
state2 = np.array([0,0,1,0])
state3 = np.array([0,0,0,1])
state22 = np.kron(state2,state2)
state33 = np.kron(state3,state3)
state22t = np.vstack(state22)
state33t = np.vstack(state33)
Hq = np.identity(16)-(1-f)*np.kron((state22 + state33)/math.sqrt(2),(state22t + state33t)/math.sqrt(2))
Hq

array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  0.  ,  0. 

In [6]:
import scipy

t = 0.5
e_iHqt = Operator(scipy.linalg.expm((0+1j)*Hq*t))
e_iHqt

Operator([[ 0.87758256+0.47942554j,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.87758256+0.47942554j,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ,
            0.        +0.j        ,  0.        +0.j        ],
          [ 0.        

In [7]:
controls = QuantumRegister(4)
circuit = QuantumCircuit(controls)
circuit.unitary(e_iHqt, [0,1,2,3], label='e^iHqt')
#circuit.measure_all()
transpiled = transpile(circuit, backend)
print(transpiled)
circuit.draw()

global phase: 5.8903
      ┌────────────┐                     ┌──────────┐  ┌──────────┐ »
q0_0: ┤ P(-2.9147) ├────────────────■────┤ Rz(-π/2) ├──┤ Ry(3π/4) ├─»
      ├────────────┤┌────────────┐┌─┴─┐┌─┴──────────┴┐┌┴──────────┴┐»
q0_1: ┤ Rz(3.1369) ├┤ Rx(1.0304) ├┤ X ├┤ Rz(0.29942) ├┤ Ry(1.1139) ├»
      └┬──────────┬┘└────────────┘└───┘└─────────────┘└────────────┘»
q0_2: ─┤ P(0.125) ├─────────────────────────────────────────────────»
      ┌┴──────────┴┐                                                »
q0_3: ┤ P(0.46875) ├────────────────────────────────────────────────»
      └────────────┘                                                »
«                          ┌──────────┐┌─────────────┐                         »
«q0_0: ─────────────────■──┤ Ry(3π/4) ├┤ Rz(-2.9877) ├──■──────────────────────»
«      ┌─────────────┐┌─┴─┐├─────────┬┘└┬────────────┤  │                      »
«q0_1: ┤ Rz(-2.1813) ├┤ X ├┤ Ry(π/2) ├──┤ Rz(1.1189) ├──┼───────────────────■──»
«      └─────────────┘└──

┌─────────┐
q0_0: ┤0        ├
      │         │
q0_1: ┤1        ├
      │  e^iHqt │
q0_2: ┤2        ├
      │         │
q0_3: ┤3        ├
      └─────────┘

In [8]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute
from qiskit import Aer

job_sim = backend.run(transpiled, shots=1)
result_sim = job_sim.result()
counts = result_sim.get_counts()
counts


IonQCredentialsError: IonQCredentialsError('Credentials `token` may not be None!')